In [ ]:
import pandas as pd
import numpy as np

# Load movie data
movies_df = pd.read_csv('movies.csv')
# Assuming movies.csv has columns 'movieId' and 'title'
movies = movies_df[['movieId', 'title']]

# Load ratings data
ratings_df = pd.read_csv('ratings.csv')
# Assuming ratings.csv has columns 'userId', 'movieId', and 'rating'
ratings = ratings_df[['userId', 'movieId', 'rating']]

# Merge movie and ratings data
df = pd.merge(ratings, movies, on='movieId')

# Create a user-item matrix
user_item_matrix = df.pivot_table(index='userId', columns='title', values='rating', fill_value=0)

# Calculate item-item similarity (cosine similarity)
item_similarity = np.dot(user_item_matrix.T, user_item_matrix) / (np.linalg.norm(user_item_matrix.T, axis=1) * np.linalg.norm(user_item_matrix, axis=0))

# Make predictions for the test set (just a basic example, you might need to adjust based on your requirements)
def predict_ratings(user_item_matrix, item_similarity, user_id):
    user_ratings = user_item_matrix.loc[user_id]
    weighted_sum = np.dot(item_similarity, user_ratings)
    norm_sum = np.dot(np.abs(item_similarity), (user_ratings > 0).astype(int))
    predicted_ratings = np.divide(weighted_sum, norm_sum, out=np.zeros_like(weighted_sum), where=norm_sum != 0)
    return predicted_ratings

# Example: Get top N recommendations for user 1
user_id = 1
predicted_ratings = predict_ratings(user_item_matrix, item_similarity, user_id)
top_n_indices = np.argsort(predicted_ratings)[::-1][:5]  # Get indices of top 5 movies
top_n_movies = user_item_matrix.columns[top_n_indices].tolist()

print(f"Top 5 recommendations for User {user_id}: {top_n_movies}")
